### NLP(Natural Language Processing)

tensorflow에서 제공하는 Tokenizer는 띄어쓰기를 기준으로 단어 인코딩 사전을 생성하고 단어를 쉽게 인코딩할 수 있게 도와준다. 

In [11]:
from keras.preprocessing.text import Tokenizer

to_token = Tokenizer()

sentences = [
    "아 배가 너무 고프다",
    "오늘 점심은 뭐먹지?",
    "햄버거 먹으러 갈까"
]

to_token.fit_on_texts(sentences) # 띄어쓰기를 기준으로 인코딩된 사전을 생성한다.
print(f"단어 인덱스 : {to_token.word_index}") # 이 때 키값이 각 토큰이며, value값이 숫자이다.

# 인코딩된 결과 살펴보기
word_encoding = to_token.texts_to_sequences(sentences)
print(word_encoding)

# 사전에 없는 단어들은?
new_sentence = ["오늘 햄버거 각이다. 아 나는 햄버거를 굉장히 좋아해"]
word_encoding1 = to_token.texts_to_sequences(new_sentence)
print(word_encoding1) # 아예 무시해버린다.

단어 인덱스 : {'아': 1, '배가': 2, '너무': 3, '고프다': 4, '오늘': 5, '점심은': 6, '뭐먹지': 7, '햄버거': 8, '먹으러': 9, '갈까': 10}
[[1, 2, 3, 4], [5, 6, 7], [8, 9, 10]]
[[5, 8, 1]]


### OOV(Out Of Vocabulary)
사전에 없는 새로운 단어들을 말한다. 케라스는 이를 처리하기 위해서 oov_token과 같은 파라미터값을 사용할 수 있다. 아래 코드의 결과를 보면 동일한 문장에서 token으로 지정한 oov를 포함한 10개의 값이 인코딩 된 것을 확인할 수 있다.

In [12]:
tokenizer = Tokenizer(oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)
word_index =  tokenizer.word_index

new_word_encoding = tokenizer.texts_to_sequences(new_sentence)

print(f"OOV가 포함된 단어 사전 {word_index}")
print(f"OOV처리된 새로운 문자열에 대한 결과 : {new_word_encoding}")

OOV가 포함된 단어 사전 {'<OOV>': 1, '아': 2, '배가': 3, '너무': 4, '고프다': 5, '오늘': 6, '점심은': 7, '뭐먹지': 8, '햄버거': 9, '먹으러': 10, '갈까': 11}
OOV처리된 새로운 문자열에 대한 결과 : [[6, 9, 1, 2, 1, 1, 1, 1]]


### 빈도수가 적은 단어는 제외하기
텍스트 데이터셋에 빈도수가 적은 단어가 많이 존재하는 경우에는 이들 단어를 제외하는 것이 일반적이다. 즉, 문장을 토큰으로 인코딩할 때 빈도수가 많은 순서대로 최대 사전 개수를 정하고 빈도수가 적은 단어를 제외한다. 최대 사전 개수는 `num_words` 파라미터를 통해 설정한다. 3으로 설정하는 경우 빈도수 순으로 3개 토큰만 인코딩하고, 나머지 단어는 "1" (OOV)로 인코딩한다.

In [13]:
tokenizer = Tokenizer(num_words = 3, oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
print(f"최대 사전 개수를 3으로 지정하여 만든 단어사전 : {word_index}") # 현재는 모든 단어가 토큰화 된 형태로 출력되지만

new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
print(f"최대 사전 개수를 3으로 지정해 산출한 결과 : {new_word_encoding}") # 이 때 최대 사전 개수가 적용되어 출력된다. {'<OOV>': 1, '아': 2, '배가': 3}
                                                                          # 따라서 최대 사전 개수가 적용된 사전 안의 "아"만 포함된 결과로 출력된다.

최대 사전 개수를 3으로 지정하여 만든 단어사전 : {'<OOV>': 1, '아': 2, '배가': 3, '너무': 4, '고프다': 5, '오늘': 6, '점심은': 7, '뭐먹지': 8, '햄버거': 9, '먹으러': 10, '갈까': 11}
최대 사전 개수를 3으로 지정해 산출한 결과 : [[1, 1, 1, 2, 1, 1, 1, 1]]


### 신경망에 넣기 위한 별도의 작업
RNN에 sequence데이터를 넣으려면 각 sequence의 길이가 동일해야 한다. CNN에서 사용했던 패딩을 똑같이 사용한다. 패딩을 적용한 모습은 아래와 같다.

```
        = pre(기본값) 0 0 2 6 4
Padding | 
        = post        2 6 4 0 0
```
keras에서 제공하는 `pad_sequence()` 함수를 이용하면 인코딩된 문장의 길이를 동일하게 만들 수 있다. 최대 문장의 길이를 기준으로 그보다 짧다면 앞에 0 값이 채워지는 `zero-padding`을 진행한다.

In [16]:
# 문장의 길이 맞추기
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(word_encoding) # 0이 붙는 위치의 기본값 = pre(앞쪽)
print(padded)

[[ 1  2  3  4]
 [ 0  5  6  7]
 [ 0  8  9 10]]


In [17]:
# 뒤쪽을 0으로 채우기
padded = pad_sequences(word_encoding, padding = "post")
print(padded)

[[ 1  2  3  4]
 [ 5  6  7  0]
 [ 8  9 10  0]]


In [18]:
# 만약 몇몇 문장만 길이가 길고, 대부분의 문장의 길이가 4 이하라고 하면, 최대 길이를 지정할 수도 있다.
padded = pad_sequences(word_encoding, padding = "post", maxlen = 3) # 기본적으로 앞에서 자른다.
print(padded)

[[ 2  3  4]
 [ 5  6  7]
 [ 8  9 10]]


In [19]:
# maxlen 적용 시 뒤에서 자르고 싶다면?
padded = pad_sequences(word_encoding, padding = "post", maxlen = 3, truncating = "post")
print(padded)

[[ 1  2  3]
 [ 5  6  7]
 [ 8  9 10]]
